|STT|	Họ và tên|	MSSV	|Email|
|-|--|--|--|
|1|	Trần Hoàng Khang	|19521671|	19521671@gm.uit.edu.vn|
|2|	Nguyễn Tú Ngọc	|20521665	|20521665@gm.uit.edu.vn|
|3 |	Lê Hồng Bằng	|19520396	|19520396@gm.uit.edu.vn|



Official documentation OpenAI Gym: https://www.gymlibrary.dev/environments/toy_text/taxi/

**Taxi-v3**: Map Correction + Cleaner Domain Description, v0.25.0 action masking added to the reset and step information

In [126]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# 1. General view

In [127]:
! pip install gym[toy_text]
! pip install stable_baselines3 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [128]:
import numpy as np
import gym
import random

from gym.wrappers import Monitor

save_video_path = '/content/drive/MyDrive/Lab/Ex2/video'

# create Taxi environment and prepare the path to store output video
env = Monitor(gym.make('Taxi-v3'), save_video_path, force=True)

In [151]:
import json

# There are many JSON file about metadata of an episodes
# There is 1 file storing the map status, you can consider it as a video

f = open('/content/video/openaigym.video.1.76.video000000.json', 'r')
file = f.read()
jsonObj = json.loads(file)
print(jsonObj['stdout'])

[[0.5, '\x1b[2J\x1b[1;1H+---------+\r\n|R: | : :\x1bG\x1b|\r\n| : | :\x1b \x1b: |\r\n| : : : : |\r\n| | : | : |\r\n|\x1bY\x1b| : |B: |\r\n+---------+\r\n\r\n'], [0.5, '\x1b[2J\x1b[1;1H+---------+\r\n|R: | : :\x1bG\x1b|\r\n| : | :\x1b \x1b: |\r\n| : : : : |\r\n| | : | : |\r\n|\x1bY\x1b| : |B: |\r\n+---------+\r\n  (Dropoff)\r\n'], [0.5, '\x1b[2J\x1b[1;1H+---------+\r\n|R: | : :\x1bG\x1b|\r\n| : | :\x1b \x1b: |\r\n| : : : : |\r\n| | : | : |\r\n|\x1bY\x1b| : |B: |\r\n+---------+\r\n  (Dropoff)\r\n'], [0.5, '\x1b[2J\x1b[1;1H+---------+\r\n|R: | : :\x1bG\x1b|\r\n| : | :\x1b \x1b: |\r\n| : : : : |\r\n| | : | : |\r\n|\x1bY\x1b| : |B: |\r\n+---------+\r\n  (Pickup)\r\n'], [0.5, '\x1b[2J\x1b[1;1H+---------+\r\n|R: | :\x1b \x1b:\x1bG\x1b|\r\n| : | : : |\r\n| : : : : |\r\n| | : | : |\r\n|\x1bY\x1b| : |B: |\r\n+---------+\r\n  (North)\r\n'], [0.5, '\x1b[2J\x1b[1;1H+---------+\r\n|R: | :\x1b \x1b:\x1bG\x1b|\r\n| : | : : |\r\n| : : : : |\r\n| | : | : |\r\n|\x1bY\x1b| : |B: |\r\n+---------+\r\n  (Nor

## Description
- There are four designated locations in the grid world indicated by `R(ed)`, `G(reen)`, `Y(ellow)`, and `B(lue)`. 
- When the episode starts, the taxi starts off at a **random square** and the passenger is at a **random location** (R/G/Y/B).
- The pipe ("|") represents a wall which the taxi cannot cross.
- The taxi drives to the passenger’s location -> *picks up the passenger*, *drives to the passenger’s destination* (another one of the four specified locations) -> *drops off the passenger*. 
- Once the passenger is dropped off -> the episode ends.

<pre>
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
</pre>

In [129]:
# Observation and action space 
obs_space = env.observation_space
action_space = env.action_space
print("The observation space: {}".format(obs_space))
print("The action space: {}".format(action_space))

The observation space: Discrete(500)
The action space: Discrete(6)


## a. Action space:

As the figure shown above, there are **6 discrete deterministic actions**:

- 0: move south

- 1: move north

- 2: move east

- 3: move west

- 4: pickup passenger

- 5: drop off passenger

## b. Observation space:

According to the rule, there are `25` taxi positions, `5` possible locations of the passenger (including the case when the passenger is in the taxi), and `4` destination locations. => **25\*5\*4 = 500 states**

Each state space is represented by the tuple: **(taxi_row, taxi_col, passenger_location, destination)**

The state tuple can then be decoded with the `decode` method from integer with the following convention:

Passenger locations:

- 0: R(ed)

- 1: G(reen)

- 2: Y(ellow)

- 3: B(lue)

- 4: in taxi

Destinations:

- 0: R(ed)

- 1: G(reen)

- 2: Y(ellow)

- 3: B(lue)

## c. Rewards rules (Read carefully)

- -1 per step unless other reward is triggered.

- +20 delivering passenger.

- -10 executing “pickup” and “drop-off” actions illegally.

In [130]:
env.reset() # Reset the state of the environment randomly

283

We would apply MDL (Markov Decision Process) model into our training. The MDL suits for discrete, stochastic, sequential environments.

<center>
 <img src="https://i.imgur.com/GeLizkh.png" alt="mdp-model">
</center>

Carry out a **stochastic** chain of actions. This should be fun when watching in your own local terminal when observing the map's changes

> Note: In the map, `yellow square` is our **taxi agent**, location (R/G/B/Y) of **passenger** is highlighted with `blue`, **destination** is `red`

In [131]:
num_steps = 99
for s in range(num_steps+1):
    print(f"step: {s} out of {num_steps}")

    # sample a random action from the list of available actions
    action = env.action_space.sample()

    # perform this action on the environment
    env.step(action)

    # print the new state
    map = env.render(mode='ansi')
    print(map)

# end this instance of the taxi environment
env.close()

step: 0 out of 99
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

step: 1 out of 99
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

step: 2 out of 99
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

step: 3 out of 99
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

step: 4 out of 99
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

step: 5 out of 99
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

step: 6 out of 99
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

step: 7 out of 99
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

step: 8 out of 99
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+----

# 2. Training and Applying (Demo step-by-step)

## Here comes the Q-Table

Reference: [RL Gym OpenAI Tutorial](https://www.gocoder.one/blog/rl-tutorial-with-openai-gym)

A Q-table is simply a look-up table storing values representing **the maximum expected future rewards** our agent can expect for a *certain action* in a *certain state* (known as `Q-values`). It will tell our agent that when it encounters a certain state, some actions are more likely than others to lead to higher rewards. It becomes a **'cheatsheet'** or you can comprehend it like a **'small database'** telling our agent what the best action to take is.

The image below illustrates what our **Q-table** will look like:

<center>
 <img src="https://i.imgur.com/e7yGkcl.png" alt="q-table">
</center>

- Each **row** corresponds to a unique `state` in the 'Taxi' environment
- Each **column** corresponds to an `action` our agent can take
- Each **cell** corresponds to the `Q-value` for that state-action pair - a higher Q-value means a higher maximum reward our agent can expect to get if it takes that action in that state. 

> Note: The `Q-values` could be achieved through **exploration** and **exploitation**. In other words, the `Q-tables` is constructed by learning the map.

So how the `Q-values` is calculated. Well, with **Q-Learning algorithm**, the following is the formula: 

<center>
 <img src="https://i.imgur.com/NOQfLd7.png" alt="Q-learning-formula">
</center>

In [132]:
# Pseudo code:
# Qlearning algorithm: Q(s,a) := Q(s,a) + learning_rate * (reward + discount_rate * max Q(s',a') - Q(s,a))

In [133]:
# First, initialize q-table with 2-d dimension array with 'state' and 'action'
state_size = env.observation_space.n
action_size = env.action_space.n
qtable = np.zeros((state_size, action_size))
qtable

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

> Note: We set some value `epsilon` between 0 and 1. If `epsilon` is **0.7**, there is a **70%** chance that on this step our agent will explore instead of exploit. `epsilon` exponentially `decays` with each step, so that our agent explores less and less over time

In [134]:
# Define hyperparameters

# hyperparameters to tune or for adjustment
learning_rate = 0.9 # how easily the agent should accept new information over previously learnt information
discount_rate = 0.8 # how much the agent should take into consideration the rewards it could receive in the future versus its immediate reward

# exploration-exploitation tradeoff
epsilon = 1.0
decay_rate= 0.005

In [135]:
# training variables
num_episodes = 1000
max_steps = 99 # per episode

Training phase

> Note: Use the code in the comment if there is an error like `ValueError: not enough values to unpack (expected 5, got 4)` due to version compatibility. See the changes [here](https://stackoverflow.com/questions/73195438/openai-gyms-env-step-what-are-the-values)

In [136]:
import random

for episode in range(num_episodes):
    print("======= Start episode %d =======" % episode)
    # reset the environment
    state = env.reset()
    done = False

    for s in range(max_steps):

        # exploration-exploitation tradeoff
        if random.uniform(0,1) < epsilon:
            # Exploration
            action = env.action_space.sample()
        else:
            # Exploitation
            action = np.argmax(qtable[state,:])

        # take action and observe reward from the current action
        #new_state, reward, terminated, truncated, info = env.step(action)
        new_state, reward, done, info = env.step(action) 
        print("New state: %d" % new_state)
        print("Reward: %d" % reward)
        print("Done: %r" % done)
        print("Info: %s" % info)

        # Q-learning algorithm
        # Update the Q-table
        qtable[state,action] = qtable[state,action] + learning_rate * (reward + discount_rate * np.max(qtable[new_state,:])-qtable[state,action])

        # Update to our new state
        state = new_state

        # if done, finish episode
        #if terminated == True or truncated == True:
        if done == True:
            break

    # Decrease epsilon
    epsilon = np.exp(-decay_rate*episode)

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print(f"TRAINING PROCESS COMPLETED OVER {num_episodes} EPISODES")

Streaming output truncated to the last 5000 lines.
New state: 279
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 379
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 479
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 475
Reward: 20
Done: True
Info: {'prob': 1.0}
======= Start episode 910 =======
New state: 371
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 271
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 251
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 231
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 211
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 311
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 411
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 419
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 319
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 219
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 239
Reward: -1
Done: False
Info: {'prob': 1.0}
New state: 259
Reward: -1
Done: False
Info: {'

> Note: Use the code in the comment if there is an error like `ValueError: not enough values to unpack (expected 5, got 4)` due to version compatibility. See the changes [here](https://stackoverflow.com/questions/73195438/openai-gyms-env-step-what-are-the-values)

In [137]:
# watch trained agent

state = env.reset()
# terminated = False
# truncated = False
done = False
rewards = 0

for s in range(max_steps):

    print(f"TRAINED AGENT")
    print("Step {}".format(s+1))

    action = np.argmax(qtable[state,:])
    #new_state, reward, terminated, truncated, info = env.step(action)
    new_state, reward, done, info = env.step(action)
    rewards += reward # Accumulate the reward

    # Show the current state with action
    map = env.render(mode='ansi')
    print(map)

    print(f"score: {rewards}")
    state = new_state

    #if terminated == True or truncated == True:
    if done == True:
        break

env.close()

TRAINED AGENT
Step 1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

score: -1
TRAINED AGENT
Step 2
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

score: -2
TRAINED AGENT
Step 3
+---------+
|R: | : :G|
|_: | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

score: -3
TRAINED AGENT
Step 4
+---------+
|R: | : :G|
| : | : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

score: -4
TRAINED AGENT
Step 5
+---------+
|R: | : :G|
| : | : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

score: -5
TRAINED AGENT
Step 6
+---------+
|R: | : :G|
| : | : : |
| : :_: : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

score: -6
TRAINED AGENT
Step 7
+---------+
|R: | : :G|
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (East)

score: -7
TRAINED AGENT
Step 8
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (South)

scor

# 3. Evaluation

a. Build model as function for reuse 

In [138]:
def q_learning(env, num_episodes, max_steps, learning_rate, discount_rate, epsilon, decay_rate):
    q_table = np.zeros((env.observation_space.n, env.action_space.n))
    rewards_all = []
    for episode in range(num_episodes):
        state = env.reset()

        reward_episode = 0.0
        done = False
        # epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate*episode)
        for step in range(max_steps):
            exploration = random.uniform(0,1)
            if exploration < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[state, :])

            next_state, reward, done, info = env.step(action) 
            q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[next_state,:]))

            reward_episode += reward
            state = next_state

            if done:
                break
        rewards_all.append(reward_episode)
    print(f'Episode {episode} finished')
    return q_table, rewards_all

In [139]:
q_table, rewards_all = q_learning(env, num_episodes, max_steps, learning_rate, discount_rate, epsilon, decay_rate)

Episode 999 finished


In [140]:
q_table

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-4.28971922, -4.16190225, -4.13513438, -4.16190225, -1.6445568 ,
        -9.        ],
       [-3.72344119, -3.47996664, -3.71689872, -3.47996664,  3.192     ,
        -9.        ],
       ...,
       [-3.14630078,  4.09758312, -3.14630078, -3.13929922, -9.        ,
        -9.        ],
       [-3.47996664, -3.51038543, -3.51333323, -3.52896892, -9.        ,
        -9.        ],
       [-1.638     , -1.638     , -1.638     , 11.97      , -9.        ,
        -9.        ]])

b. Create `play` function to watch trained agent: Reset the environment and do the 'watch the trained agent' task above

In [141]:
def play(env, q_table, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = np.argmax(q_table[state, :])
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

c. Show performance: If reward is positve, we win this case

In [142]:
def play_multiple_times(env, q_table, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, q_table)

        if total_reward > 0: 
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [143]:
play_multiple_times(env, q_table, 1000)

Number of successes: 989/1000
Average number of steps: 13.180990899898887
